In [1]:
import os
os.chdir("../")

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


k_fold = 10
seeds = [192760, 1393659, 1269293, 138973, 931308, 1088652, 1361151, 1456105, 820535, 1240839]


In [3]:
data_train_path = "Dataset/data_train.csv"
data_test_path = "Dataset/data_target_users_test.csv"

In [4]:
df = pd.read_csv(data_train_path)
df_test = pd.read_csv(data_test_path)

df.columns = ["UserID", "ItemID", "Interaction"]
df_test.columns = ["UserID"]

In [5]:
df

,UserID,ItemID,Interaction
0,1,7,1.0
1,1,15,1.0
2,1,16,1.0
3,1,133,1.0
4,1,161,1.0
...,...,...,...
478725,13024,13605,1.0
478726,13024,13823,1.0
478727,13024,15122,1.0
478728,13024,18185,1.0


In [6]:
metric_to_optimize="MAP"
cutoff_to_optimize=10

In [7]:
user_ids = df["UserID"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
item_ids = df["ItemID"].unique().tolist()
item2item_encoded = {x: i for i, x in enumerate(item_ids)}
item_encoded2item = {i: x for i, x in enumerate(item_ids)}
df["User"] = df["UserID"].map(user2user_encoded)
df["Item"] = df["ItemID"].map(item2item_encoded)

num_users = len(user2user_encoded)
num_items = len(item_encoded2item)
df["Interaction"] = df["Interaction"].values.astype(np.float32)

# min and max ratings will be used to normalize the ratings later
min_rating = 0.0
max_rating = max(df["Interaction"])

print(
    "Number of users: {}, Number of Items: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_items, min_rating, max_rating
    )
)

Number of users: 12638, Number of Items: 22222, Min rating: 0.0, Max rating: 1.0


In [8]:
userId_unique = df["UserID"].unique()
itemId_unique = df["ItemID"].unique()

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample


urm_all = sps.coo_matrix((df["Interaction"].values, 
                          (df["User"].values, df["Item"].values)))

urm_train_validation = []
urm_test = []
urm_train = []
urm_validation = []

for i in range(len(seeds)):
    a, b = split_train_in_two_percentage_global_sample(urm_all, train_percentage = 0.80,seed=seeds[i])
    c, d = split_train_in_two_percentage_global_sample(a, train_percentage = 0.80,seed=seeds[i])
    urm_train_validation.append(a)
    urm_test.append(b)
    urm_train.append(c)
    urm_validation.append(d)



In [10]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = []
for u in urm_validation:
    evaluator_validation.append(EvaluatorHoldout(u, cutoff_list=[10], ignore_users=[]))

EvaluatorHoldout: Ignoring 2581 (20.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2598 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2621 (20.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2601 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2649 (21.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2614 (20.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2608 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2586 (20.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2593 

In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = []
for u in urm_test:
    evaluator_test.append(EvaluatorHoldout(u, cutoff_list=[10], ignore_users=[]))

EvaluatorHoldout: Ignoring 2146 (17.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2177 (17.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2157 (17.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2132 (16.9%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2149 (17.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2129 (16.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2228 (17.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2183 (17.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2140 

In [12]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

recommenderClass = RP3betaRecommender

## Hyperparameter Tuning

In [13]:
import optuna

# Replace the following with your MySQL database connection details
host = "db-mysql-nyc3-14396-do-user-15539286-0.c.db.ondigitalocean.com"
port = 25060
database_name = "recsys_optuna"
username = "doadmin"
password = "AVNS_EIQ1D-GbDGJWZPsN3Mt"

# Create an SQLAlchemy engine
mysql_url = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_name}"
storage = optuna.storages.RDBStorage(url=mysql_url)

In [14]:
study = optuna.create_study(study_name="RP3_Hybrid_last", direction="maximize", storage=storage, load_if_exists=True)

[I 2024-01-09 22:08:03,151] A new study created in RDB with name: RP3_Hybrid_last


In [15]:
def objective(trial):

    params = {
        'topK': trial.suggest_int('topK', 5, 1000),
        'alpha': trial.suggest_float('alpha', 0, 2),
        'beta': trial.suggest_float('beta', 0, 2),
    }

    MAP = 0
    for i in range(k_fold):
        recommender = recommenderClass(urm_train[i])
        recommender.fit(**params)

        result_dict, _ = evaluator_validation[i].evaluateRecommender(recommender)

        MAP += result_dict["MAP"].item()
    
    MAP /= k_fold
    return MAP

In [17]:
study.optimize(objective, n_trials=200)

RP3betaRecommender: URM Detected 508 ( 4.0%) users with no interactions.
RP3betaRecommender: URM Detected 332 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 3625.81 column/sec. Elapsed time 6.13 sec
EvaluatorHoldout: Processed 10057 (100.0%) in 6.64 sec. Users per second: 1514
RP3betaRecommender: URM Detected 440 ( 3.5%) users with no interactions.
RP3betaRecommender: URM Detected 325 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 4608.18 column/sec. Elapsed time 4.82 sec
EvaluatorHoldout: Processed 10040 (100.0%) in 5.32 sec. Users per second: 1889
RP3betaRecommender: URM Detected 479 ( 3.8%) users with no interactions.
RP3betaRecommender: URM Detected 345 ( 1.6%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 4584.17 column/sec. Elapsed time 4.85 sec
EvaluatorHoldout: Processed 10017 (100.0%) in 6.06 sec. Users per second: 1653
RP3betaRecommender: URM Detected 451 

[I 2024-01-10 00:20:18,768] Trial 149 finished with value: 0.029091687251580796 and parameters: {'topK': 18, 'alpha': 0.37105794586906055, 'beta': 0.29100602536905507}. Best is trial 103 with value: 0.029994871944712682.


RP3betaRecommender: URM Detected 508 ( 4.0%) users with no interactions.
RP3betaRecommender: URM Detected 332 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 6048.29 column/sec. Elapsed time 3.67 sec
EvaluatorHoldout: Processed 10057 (100.0%) in 4.55 sec. Users per second: 2208
RP3betaRecommender: URM Detected 440 ( 3.5%) users with no interactions.
RP3betaRecommender: URM Detected 325 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 5441.62 column/sec. Elapsed time 4.08 sec
EvaluatorHoldout: Processed 10040 (100.0%) in 4.78 sec. Users per second: 2100
RP3betaRecommender: URM Detected 479 ( 3.8%) users with no interactions.
RP3betaRecommender: URM Detected 345 ( 1.6%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 4088.90 column/sec. Elapsed time 5.43 sec
EvaluatorHoldout: Processed 10017 (100.0%) in 5.26 sec. Users per second: 1904
RP3betaRecommender: URM Detected 451 

[I 2024-01-10 00:22:08,154] Trial 152 finished with value: 0.02038295553895023 and parameters: {'topK': 5, 'alpha': 0.35552165025286286, 'beta': 0.05066873402388157}. Best is trial 103 with value: 0.029994871944712682.


RP3betaRecommender: URM Detected 508 ( 4.0%) users with no interactions.
RP3betaRecommender: URM Detected 332 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 3968.63 column/sec. Elapsed time 5.60 sec
EvaluatorHoldout: Processed 10057 (100.0%) in 8.65 sec. Users per second: 1163
RP3betaRecommender: URM Detected 440 ( 3.5%) users with no interactions.
RP3betaRecommender: URM Detected 325 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 3195.62 column/sec. Elapsed time 6.95 sec
EvaluatorHoldout: Processed 10040 (100.0%) in 4.91 sec. Users per second: 2045
RP3betaRecommender: URM Detected 479 ( 3.8%) users with no interactions.
RP3betaRecommender: URM Detected 345 ( 1.6%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 3619.70 column/sec. Elapsed time 6.14 sec
EvaluatorHoldout: Processed 10017 (100.0%) in 5.96 sec. Users per second: 1680
RP3betaRecommender: URM Detected 451 

...

[I 2024-01-10 07:07:55,632] Trial 761 finished with value: 0.029727513050698214 and parameters: {'topK': 38, 'alpha': 0.2877002866925834, 'beta': 0.2529982972361287}. Best is trial 549 with value: 0.030056746039487436.


RP3betaRecommender: URM Detected 508 ( 4.0%) users with no interactions.
RP3betaRecommender: URM Detected 332 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 4660.92 column/sec. Elapsed time 4.77 sec
EvaluatorHoldout: Processed 10057 (100.0%) in 3.69 sec. Users per second: 2728
RP3betaRecommender: URM Detected 440 ( 3.5%) users with no interactions.
RP3betaRecommender: URM Detected 325 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 5467.80 column/sec. Elapsed time 4.06 sec
EvaluatorHoldout: Processed 10040 (100.0%) in 4.11 sec. Users per second: 2442
RP3betaRecommender: URM Detected 479 ( 3.8%) users with no interactions.
RP3betaRecommender: URM Detected 345 ( 1.6%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 3861.34 column/sec. Elapsed time 5.75 sec
EvaluatorHoldout: Processed 10017 (100.0%) in 4.75 sec. Users per second: 2109
RP3betaRecommender: URM Detected 451 

[I 2024-01-10 07:09:55,314] Trial 762 finished with value: 0.029407934691964815 and parameters: {'topK': 67, 'alpha': 0.36867216286844456, 'beta': 0.23404690307750986}. Best is trial 549 with value: 0.030056746039487436.


RP3betaRecommender: URM Detected 508 ( 4.0%) users with no interactions.
RP3betaRecommender: URM Detected 332 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 3943.80 column/sec. Elapsed time 5.63 sec
EvaluatorHoldout: Processed 10057 (100.0%) in 4.77 sec. Users per second: 2109
RP3betaRecommender: URM Detected 440 ( 3.5%) users with no interactions.
RP3betaRecommender: URM Detected 325 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 4198.13 column/sec. Elapsed time 5.29 sec
EvaluatorHoldout: Processed 10040 (100.0%) in 4.26 sec. Users per second: 2358
RP3betaRecommender: URM Detected 479 ( 3.8%) users with no interactions.
RP3betaRecommender: URM Detected 345 ( 1.6%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 5095.03 column/sec. Elapsed time 4.36 sec
EvaluatorHoldout: Processed 10017 (100.0%) in 2.62 sec. Users per second: 3824
RP3betaRecommender: URM Detected 451 

[I 2024-01-10 07:11:49,964] Trial 767 finished with value: 0.029950274012610256 and parameters: {'topK': 48, 'alpha': 0.26067357041643724, 'beta': 0.1467660893023432}. Best is trial 549 with value: 0.030056746039487436.


## Fit and Test

In [18]:
best_MAP_test = 0

for i in range(k_fold):
    recommender = recommenderClass(urm_train[i])
    recommender.fit(**study.best_params)

    result_dict, _ = evaluator_test[i].evaluateRecommender(recommender)

    best_MAP_test += result_dict["MAP"].item()

best_MAP_test /= k_fold
best_MAP_test

RP3betaRecommender: URM Detected 508 ( 4.0%) users with no interactions.
RP3betaRecommender: URM Detected 332 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 4459.64 column/sec. Elapsed time 4.98 sec
EvaluatorHoldout: Processed 10492 (100.0%) in 5.47 sec. Users per second: 1916
RP3betaRecommender: URM Detected 440 ( 3.5%) users with no interactions.
RP3betaRecommender: URM Detected 325 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 4058.76 column/sec. Elapsed time 5.48 sec
EvaluatorHoldout: Processed 10461 (100.0%) in 5.11 sec. Users per second: 2049
RP3betaRecommender: URM Detected 479 ( 3.8%) users with no interactions.
RP3betaRecommender: URM Detected 345 ( 1.6%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 4987.37 column/sec. Elapsed time 4.46 sec
EvaluatorHoldout: Processed 10481 (100.0%) in 4.75 sec. Users per second: 2204
RP3betaRecommender: URM Detected 451 

0.03785676414574286

In [19]:
best_MAP_test = 0

for i in range(k_fold):
    recommender = recommenderClass(urm_train_validation[i])
    recommender.fit(**study.best_params)

    result_dict, _ = evaluator_test[i].evaluateRecommender(recommender)

    best_MAP_test += result_dict["MAP"].item()

best_MAP_test /= k_fold
best_MAP_test

RP3betaRecommender: URM Detected 249 ( 2.0%) users with no interactions.
RP3betaRecommender: URM Detected 113 ( 0.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 4245.62 column/sec. Elapsed time 5.23 sec
EvaluatorHoldout: Processed 10492 (100.0%) in 6.54 sec. Users per second: 1605
RP3betaRecommender: URM Detected 191 ( 1.5%) users with no interactions.
RP3betaRecommender: URM Detected 101 ( 0.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 4200.65 column/sec. Elapsed time 5.29 sec
EvaluatorHoldout: Processed 10461 (100.0%) in 4.01 sec. Users per second: 2609
RP3betaRecommender: URM Detected 235 ( 1.9%) users with no interactions.
RP3betaRecommender: URM Detected 105 ( 0.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 4302.18 column/sec. Elapsed time 5.17 sec
EvaluatorHoldout: Processed 10481 (100.0%) in 5.20 sec. Users per second: 2015
RP3betaRecommender: URM Detected 218 

0.0486783777948812